In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
scaler = StandardScaler()
import pandas_profiling
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score as geo
from imblearn.metrics import make_index_balanced_accuracy as iba
from sklearn.metrics import roc_curve, auc
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from pycaret.regression import *
import sqlite3
from sqlite3 import Error
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_selection import SelectKBest, mutual_info_regression, VarianceThreshold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [2]:
df1=pd.read_csv('train.csv')

In [3]:
df2=pd.read_csv('test.csv')

In [4]:
df1.info(40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55423856 entries, 0 to 55423855
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 3.3+ GB


In [5]:
df1

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
55423851,2014-03-15 03:28:00.00000070,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1
55423852,2009-03-24 20:46:20.0000002,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1
55423853,2011-04-02 22:04:24.0000004,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1
55423854,2011-10-26 05:57:51.0000002,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1


In [6]:
def datetime_info(df):
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'],format="%Y-%m-%d %H:%M:%S UTC")
    df['hour'] = df.pickup_datetime.dt.hour
    df['day'] = df.pickup_datetime.dt.day
    df['month'] = df.pickup_datetime.dt.month
    df['weekday'] = df.pickup_datetime.dt.weekday
    df['year'] = df.pickup_datetime.dt.year
    df.drop(['pickup_datetime'],axis=1,inplace=True)
    
    return df

In [7]:
datetime_info(df1)

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,weekday,year
0,2009-06-15 17:26:21.0000001,4.5,-73.844311,40.721319,-73.841610,40.712278,1,17,15,6,0,2009
1,2010-01-05 16:52:16.0000002,16.9,-74.016048,40.711303,-73.979268,40.782004,1,16,5,1,1,2010
2,2011-08-18 00:35:00.00000049,5.7,-73.982738,40.761270,-73.991242,40.750562,2,0,18,8,3,2011
3,2012-04-21 04:30:42.0000001,7.7,-73.987130,40.733143,-73.991567,40.758092,1,4,21,4,5,2012
4,2010-03-09 07:51:00.000000135,5.3,-73.968095,40.768008,-73.956655,40.783762,1,7,9,3,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
55423851,2014-03-15 03:28:00.00000070,14.0,-74.005272,40.740027,-73.963280,40.762555,1,3,15,3,5,2014
55423852,2009-03-24 20:46:20.0000002,4.2,-73.957784,40.765530,-73.951640,40.773959,1,20,24,3,1,2009
55423853,2011-04-02 22:04:24.0000004,14.1,-73.970505,40.752325,-73.960537,40.797342,1,22,2,4,5,2011
55423854,2011-10-26 05:57:51.0000002,28.9,-73.980901,40.764629,-73.870605,40.773963,1,5,26,10,2,2011


In [8]:
datetime_info(df2)

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,weekday,year
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,13,27,1,1,2015
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,13,27,1,1,2015
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,11,8,10,5,2011
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,21,1,12,5,2012
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,21,1,12,5,2012
...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,12,10,5,6,2015
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,17,12,1,0,2015
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,20,19,4,6,2015
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,1,31,1,5,2015


In [9]:
df1.drop(columns=['key'], inplace=True)
idCol = df2.key.to_numpy()
df2.drop(columns=['key'], inplace=True)
yTrain = df1.fare_amount
df1.drop(columns=['fare_amount'], inplace=True)
xTest = df2.to_numpy()
xTrain = df1.to_numpy()
xTrain.shape, yTrain.shape, xTest.shape

((55423856, 10), (55423856,), (9914, 10))

In [10]:
def SMAPE(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [11]:
from lightgbm import LGBMRegressor

lgbmRegressor = LGBMRegressor()
lgbmRegressor.fit(xTrain, yTrain)
lgbmError = SMAPE(yTrain, lgbmRegressor.predict(xTrain))
lgbmError

21.38550805301174

In [12]:
final = pd.DataFrame(columns=["key","fare_amount"])
final["key"] = idCol
final["fare_amount"] = lgbmRegressor.predict(xTest)
final

,key,fare_amount
0,2015-01-27 13:08:24.0000002,8.711677
1,2015-01-27 13:08:24.0000003,9.337318
2,2011-10-08 11:53:44.0000002,6.797032
3,2012-12-01 21:12:12.0000002,8.063781
4,2012-12-01 21:12:12.0000003,14.568108
...,...,...
9909,2015-05-10 12:37:51.0000002,8.516063
9910,2015-01-12 17:05:51.0000001,9.743922
9911,2015-04-19 20:44:15.0000001,53.447230
9912,2015-01-31 01:05:19.0000005,19.984122


In [13]:
final.to_csv('submission.csv', index=False)
print('Submission succesful!')

Submission succesful!
